### Set Up

#### Standard library imports

In [ ]:
import os
import datetime

#### Third party imports 

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

#### Local imports

In [ ]:
import modules.plotting as plt

#### Autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

#### Setting relative directories

In [ ]:
Epoch_number = 512
Epoch = "epoch_" + str(Epoch_number)
Date  = "20210720"

In [ ]:
Working_Directory = os.getcwd()
Project_Directory = os.path.abspath(os.path.join(Working_Directory,'..'))
Music_In_Directory = Project_Directory + "/data/chopin_midi/" 
Output_Directory = Project_Directory + "/outputs/"
Model_Directory = Output_Directory + "models/"
Numpy_Directory = Model_Directory + "arrays/"
Checkpoint_Directory = Model_Directory + "ckpt/"
Checkpoint_Date_Directory = Checkpoint_Directory + Date + "/"
Checkpoint_Date_Epoch_Directory = Checkpoint_Date_Directory + Epoch + "_model"

### Load model

In [ ]:
load_path = Checkpoint_Date_Epoch_Directory
#model = tf.keras.models.load_model(load_path)

#### Load Numpy arrays

In [ ]:
load_path = Numpy_Directory + Date + '/' + Epoch + '_array.npz'
npzfile = np.load(load_path) 
keys = ['train_loss_p_a', 
        'train_loss_vel',
        'train_metric_p_a', 
        'train_metric_vel', 
        'val_loss_p_a', 
        'val_loss_vel', 
        'val_metric_p_a', 
        'val_metric_vel', 
        'train_p_conf_array', 
        'train_a_conf_array',
        'val_p_conf_array', 
        'val_a_conf_array',
        'X_train', 
        'y_train', 
        'y_pred_train', 
        'X_val', 
        'y_val', 
        'y_pred_val'
       ]
np_arrays = {}
for i in range(len(keys)):
    np_arrays[keys[i]] = npzfile['arr_'+ str(i)] 

#### Compare precitions to labels

In [ ]:
note_idx = 30
time_idx = 60
batch_idx = 0

print_labels = False # True
if print_labels:
    tmp = tfp.distributions.Bernoulli(logits=np_arrays['y_pred_train'][batch_idx,:,time_idx,0:2]).sample()
    tmp2 = tf.cast(np_arrays['y_pred_train'][batch_idx,:, time_idx,2:3], dtype=tf.int32)
    tmp3 = tf.concat([tmp,tmp2], axis=-1)
    print(tf.concat([np_arrays['y_train'][batch_idx,:, time_idx,:], tmp3], -1))

### Visualiation

#### Losses $L_1$ and $L_2$

In [ ]:
fig = plt.lineplot(np_arrays, 'loss_p_a', 'Play/Articulate Loss', 'L_1', Epoch_number)
fig.show()

In [ ]:
fig = plt.lineplot(np_arrays, 'loss_vel', 'Velocity Loss', 'L_2', Epoch_number)
fig.show()

#### Accuracy

In [ ]:
fig = plt.lineplot(np_arrays, 'metric_p_a', 'Binary Accuracy', 'Play/Articulate', Epoch_number)
fig.show()

#### Confusion Matrix

In [ ]:
fig = plt.plot_confusion_matrix(np_arrays, 'train_p_conf_array', 'Played', 'Training: ', Epoch_number)
fig.show()

In [ ]:
fig = plt.plot_confusion_matrix(np_arrays, 'val_p_conf_array', 'Played', 'Validation: ', Epoch_number)
fig.show()

In [ ]:
fig = plt.plot_confusion_matrix(np_arrays, 'train_a_conf_array', 'Articulated', 'Training: ', Epoch_number)
fig.show()

In [ ]:
fig = plt.plot_confusion_matrix(np_arrays, 'val_a_conf_array', 'Articulated', 'Validation: ', Epoch_number)
fig.show()

#### Alpha anaysis

In [ ]:
Epoch_number = 32
alpha = [1.0, 0.1, 0.01, 0.001]
Epoch = "epoch_" + str(Epoch_number)
Folder  = "alpha_beta"

In [ ]:
keys = ['train_loss_p_a', 
        'train_loss_vel',
        'train_metric_p_a', 
        'train_metric_vel_array',
        'val_loss_p_a', 
        'val_loss_vel', 
        'val_metric_p_a', 
       ]
np_arrays = {}
for i in range(len(alpha)):
    for j in range(len(keys)):
        name = "alpha_" + str(alpha[i]) + "_beta_1.0_epoch_32"
        model_name = "alpha_" + str(alpha[i]) + "_beta_1.0_epoch_32" + "_model"
        array_name = "alpha_" + str(alpha[i]) + "_beta_1.0_epoch_32" + "_array.npz"
        load_path = Numpy_Directory + Folder + '/' + array_name
        npzfile = np.load(load_path) 
        key_new = "alpha_" + str(alpha[i]) + "_" + keys[j]
        np_arrays[key_new] = npzfile['arr_' + str(j)] 

In [ ]:
fig = plt.lineplot_alphas(np_arrays, 'loss_vel', 'Velocity Loss', 'L_2', Epoch_number)
fig.show()

In [ ]:
fig = plt.lineplot_alphas(np_arrays, 'loss_p_a', 'Play/Articulate Loss', 'L_1', Epoch_number)
fig.show()